<a href="https://colab.research.google.com/github/cedamusk/fun-projects/blob/main/Cvs_And_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install torch torchvision transformers pillow


In [9]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification
from sklearn.model_selection import train_test_split
from google.colab import drive

In [10]:
def mount_google_drive():
  drive.mount('/content/drive')

In [11]:
def check_directory_contents(path):
  for item in os.listdir(path):
    print(item)

In [12]:
mount_google_drive()
dataset_path='/content/drive/MyDrive/minet'
check_directory_contents(dataset_path)

Mounted at /content/drive
biotite
pyrite
muscovite
bornite
malachite
quartz
chrysocolla


In [13]:
minerals=['biotite', 'pyrite', 'muscovite', 'bornite', 'malachite', 'quartz', 'chrysocolla']

In [14]:
image_paths=[]
labels=[]
for idx, mineral in enumerate(minerals):
  mineral_dir=f"/content/drive/MyDrive/minet"
  if os.path.exists(mineral_dir):
    for img_name in os.listdir(mineral_dir):
      image_paths.append(os.path.join(mineral_dir, img_name))
      labels.append(idx)

In [15]:
train_paths, val_paths, train_labels, val_labels=train_test_split(image_paths, labels, test_size=0.2, random_state=42)

In [16]:
class MineralDataset(Dataset):
  def __init__(self, image_paths, labels, transform=None):
    self.image_paths=image_paths
    self.labels=labels
    self.transform=transform

  def __len__(self):
    return len(self.image_paths)

  def __getitem__(self, idx):
    image=Image.open(self.image_paths[idx]).convert('RGB')
    label=self.labels[idx]
    if self.transform:
      image=self.transform(image)
    return image, label

In [17]:
transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [18]:
train_dataset=MineralDataset(train_paths, train_labels, transform=transform)
val_dataset=MineralDataset(val_paths, val_labels, transform=transform)

In [20]:
model_name='google/vit-base-patch16-224-in21k'
feature_extractor=ViTFeatureExtractor.from_pretrained(model_name)
model=ViTForImageClassification.from_pretrained(
    model_name, num_labels=len(minerals)
)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
